In [1]:
import pandas as pd; import numpy as np; import janitor as jn; from pandas import Timedelta

In [2]:
df = pd.DataFrame(dict(B=[2.0,0.0]))
right=pd.DataFrame(dict(Numeric=[1.0]))

In [3]:
df

,B
0,2.0
1,0.0


In [4]:
right

,Numeric
0,1.0


In [5]:
df[["B"]].conditional_join(
        right[["Numeric"]],
        ("B", "Numeric", "!="),
        how="inner",
        keep="last",

        sort_by_appearance=False,
    )

,B,Numeric
0,0.0,1.0
1,2.0,1.0


In [6]:
from string import ascii_lowercase
np.random.seed(1)
n = 2000; k = 300
mapp = {k:v for v,k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high = 100, size = n)
idx2 = np.random.randint(0, high = 100, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2),
          )


d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [7]:
d1

,x,start,end,xx
0,c,37,61,2
1,d,12,16,3
2,b,72,79,1
3,c,9,74,2
4,d,75,76,3
...,...,...,...,...
1995,e,7,27,4
1996,d,18,29,3
1997,e,51,89,4
1998,c,72,77,2


In [8]:
d2

,x,pos1,xx
0,f,143,5
1,m,106,12
2,o,114,14
3,a,132,0
4,n,125,13
...,...,...,...
295,d,122,3
296,b,109,1
297,k,103,10
298,j,72,9


In [9]:
d2.sort_values(['pos1','xx'])

,x,pos1,xx
171,b,60,1
174,f,61,5
74,l,61,11
113,m,61,12
71,n,61,13
...,...,...,...
268,f,149,5
277,i,149,8
39,a,150,0
91,a,150,0


In [10]:
bla = (d1
.conditional_join(
    d2, 
    ('start', 'pos1', '>='), 
    # ('end', 'pos1', '>='), 
    ('xx', 'xx', '<'), 
    use_numba=True)
)

bla

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


left              right         
         x start end xx     x pos1  xx
0        b    72  79  1     j   72   9
1        b    72  79  1     o   72  14
2        b    72  79  1     e   72   4
3        b    72  79  1     e   72   4
4        b    72  79  1     c   71   2
...    ...   ...  .. ..   ...  ...  ..
10067    c    72  77  2     n   62  13
10068    c    72  77  2     f   61   5
10069    c    72  77  2     m   61  12
10070    c    72  77  2     l   61  11
10071    c    72  77  2     n   61  13

[10072 rows x 7 columns]

In [11]:
(d1
.conditional_join(
    d2, 
    ('start', 'pos1', '>='), 
  #  ('end', 'pos1', '>='), 
    ('xx', 'xx', '<'), 
    use_numba=False)
)

left              right         
         x start end xx     x pos1  xx
0        b    72  79  1     n   61  13
1        b    72  79  1     l   61  11
2        b    72  79  1     m   61  12
3        b    72  79  1     f   61   5
4        b    72  79  1     n   62  13
...    ...   ...  .. ..   ...  ...  ..
10067    c    72  77  2     d   71   3
10068    c    72  77  2     e   72   4
10069    c    72  77  2     e   72   4
10070    c    72  77  2     o   72  14
10071    c    72  77  2     j   72   9

[10072 rows x 7 columns]

In [12]:
# pprint([region_left, region_right, left_region, right_region])

In [13]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})
                    
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [14]:
# df1 = pd.concat([df1]*10)
# df2 = pd.concat([df2]*10)

In [15]:
(df1
.conditional_join(
    df2,
    ('id', 'id', '>'),
    ('value_1', 'value_2A', '<='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))

left         right                  
    id value_1    id value_2A value_2B
0    2       1     1       12       15
1    2       1     1        7        9
2    2       1     1        3        5
3    2       3     1       12       15
4    2       3     1        7        9
5    2       3     1        3        5
6    3       4     1       12       15
7    3       4     1        7        9

In [16]:
df1 = pd.concat([df1]*2)
df2 = pd.concat([df2]*2)

In [17]:
df2.sort_values(['value_2A','value_2B'],kind='stable')

,id,value_2A,value_2B
0,1,0,1
4,2,0,1
0,1,0,1
4,2,0,1
7,3,1,3
7,3,1,3
5,2,2,4
5,2,2,4
1,1,3,5
1,1,3,5


In [18]:
bla = (df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
     ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))

bla

left         right                  
     id value_1    id value_2A value_2B
0     1       2     2        2        4
1     1       2     2        2        4
2     1       2     3        1        3
3     1       2     3        1        3
4     1       5     2        3        6
5     1       5     1        3        5
6     1       5     2        3        6
7     1       5     1        3        5
8     1       7     1        7        9
9     1       7     1        7        9
10    2       1     3        1        3
11    2       1     3        1        3
12    2       1     2        0        1
13    2       1     1        0        1
14    2       1     2        0        1
15    2       1     1        0        1
16    2       3     2        3        6
17    2       3     1        3        5
18    2       3     2        3        6
19    2       3     1        3        5
20    2       3     2        2        4
21    2       3     2        2        4
22    2       3     3        1        3
23    2       3     3        1        3
24    3       4     2        3        6
25    3       4     1        3        5
26    3       4     2        3        6
27    3       4     1        3        5
28    3       4     2        2        4
29    3       4     2        2        4
30    1       2     2        2        4
31    1       2     2        2        4
32    1       2     3        1        3
33    1       2     3        1        3
34    1       5     2        3        6
35    1       5     1        3        5
36    1       5     2        3        6
37    1       5     1        3        5
38    1       7     1        7        9
39    1       7     1        7        9
40    2       1     3        1        3
41    2       1     3        1        3
42    2       1     2        0        1
43    2       1     1        0        1
44    2       1     2        0        1
45    2       1     1        0        1
46    2       3     2        3        6
47    2       3     1        3        5
48    2       3     2        3        6
49    2       3     1        3        5
50    2       3     2        2        4
51    2       3     2        2        4
52    2       3     3        1        3
53    2       3     3        1        3
54    3       4     2        3        6
55    3       4     1        3        5
56    3       4     2        3        6
57    3       4     1        3        5
58    3       4     2        2        4
59    3       4     2        2        4

In [19]:
%%timeit
(df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
    # ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))


1.12 ms ± 3.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%%timeit
(df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
    # ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=False
))

892 µs ± 3.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
(df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
     ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=False
))

left         right                  
     id value_1    id value_2A value_2B
0     1       2     3        1        3
1     1       2     3        1        3
2     1       2     2        2        4
3     1       2     2        2        4
4     1       5     1        3        5
5     1       5     2        3        6
6     1       5     1        3        5
7     1       5     2        3        6
8     1       7     1        7        9
9     1       7     1        7        9
10    2       1     1        0        1
11    2       1     2        0        1
12    2       1     1        0        1
13    2       1     2        0        1
14    2       1     3        1        3
15    2       1     3        1        3
16    2       3     3        1        3
17    2       3     3        1        3
18    2       3     2        2        4
19    2       3     2        2        4
20    2       3     1        3        5
21    2       3     2        3        6
22    2       3     1        3        5
23    2       3     2        3        6
24    3       4     2        2        4
25    3       4     2        2        4
26    3       4     1        3        5
27    3       4     2        3        6
28    3       4     1        3        5
29    3       4     2        3        6
30    1       2     3        1        3
31    1       2     3        1        3
32    1       2     2        2        4
33    1       2     2        2        4
34    1       5     1        3        5
35    1       5     2        3        6
36    1       5     1        3        5
37    1       5     2        3        6
38    1       7     1        7        9
39    1       7     1        7        9
40    2       1     1        0        1
41    2       1     2        0        1
42    2       1     1        0        1
43    2       1     2        0        1
44    2       1     3        1        3
45    2       1     3        1        3
46    2       3     3        1        3
47    2       3     3        1        3
48    2       3     2        2        4
49    2       3     2        2        4
50    2       3     1        3        5
51    2       3     2        3        6
52    2       3     1        3        5
53    2       3     2        3        6
54    3       4     2        2        4
55    3       4     2        2        4
56    3       4     1        3        5
57    3       4     2        3        6
58    3       4     1        3        5
59    3       4     2        3        6

In [31]:
url='/Users/samuel.oranyeli/events.csv'
#url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
events = pd.read_csv(url, parse_dates=['start', 'end'])
events.head()

,id,name,audience,start,sponsor,end
0,1,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,2,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,3,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,4,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,5,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [23]:
bla =(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

bla

left                                          right  \
         id               start                 end     id   
0        10 1993-11-27 12:00:00 1993-11-27 12:37:00   2345   
1        15 1993-04-04 16:00:00 1993-04-04 18:00:00  11178   
2        17 2030-10-25 07:00:00 2030-10-25 07:27:00  19605   
3        26 2005-10-04 17:00:00 2005-10-04 17:18:00   8218   
4        35 2024-05-02 15:00:00 2024-05-02 15:35:00  27696   
...     ...                 ...                 ...    ...   
3697  29966 2000-08-26 11:00:00 2000-08-26 13:00:00  29375   
3698  29971 2018-05-18 04:00:00 2018-05-18 04:18:00  24173   
3699  29978 1992-06-07 22:00:00 1992-06-07 22:23:00    981   
3700  29984 2025-06-05 03:00:00 2025-06-05 03:17:00  19051   
3701  29995 2016-09-04 14:00:00 2016-09-04 14:32:00  12296   

                                              
                   start                 end  
0    1993-11-27 10:00:00 1993-11-27 12:00:00  
1    1993-04-04 17:00:00 1993-04-04 17:22:00  
2    2030-10-25 06:00:00 2030-10-25 08:00:00  
3    2005-10-04 17:00:00 2005-10-04 17:27:00  
4    2024-05-02 15:00:00 2024-05-02 15:07:00  
...                  ...                 ...  
3697 2000-08-26 13:00:00 2000-08-26 13:53:00  
3698 2018-05-18 04:00:00 2018-05-18 04:36:00  
3699 1992-06-07 22:00:00 1992-06-07 22:30:00  
3700 2025-06-05 01:00:00 2025-06-05 03:00:00  
3701 2016-09-04 14:00:00 2016-09-04 14:50:00  

[3702 rows x 6 columns]

In [24]:
events.shape[0]**2

899940001

In [32]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)


19.3 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)


19 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

left                                          right  \
         id               start                 end     id   
0        10 1993-11-27 12:00:00 1993-11-27 12:37:00   2345   
1        15 1993-04-04 16:00:00 1993-04-04 18:00:00  11178   
2        17 2030-10-25 07:00:00 2030-10-25 07:27:00  19605   
3        26 2005-10-04 17:00:00 2005-10-04 17:18:00   8218   
4        35 2024-05-02 15:00:00 2024-05-02 15:35:00   6916   
...     ...                 ...                 ...    ...   
3697  29966 2000-08-26 11:00:00 2000-08-26 13:00:00  29375   
3698  29971 2018-05-18 04:00:00 2018-05-18 04:18:00  24173   
3699  29978 1992-06-07 22:00:00 1992-06-07 22:23:00    981   
3700  29984 2025-06-05 03:00:00 2025-06-05 03:17:00  19051   
3701  29995 2016-09-04 14:00:00 2016-09-04 14:32:00  12296   

                                              
                   start                 end  
0    1993-11-27 10:00:00 1993-11-27 12:00:00  
1    1993-04-04 17:00:00 1993-04-04 17:22:00  
2    2030-10-25 06:00:00 2030-10-25 08:00:00  
3    2005-10-04 17:00:00 2005-10-04 17:27:00  
4    2024-05-02 15:00:00 2024-05-02 15:36:00  
...                  ...                 ...  
3697 2000-08-26 13:00:00 2000-08-26 13:53:00  
3698 2018-05-18 04:00:00 2018-05-18 04:36:00  
3699 1992-06-07 22:00:00 1992-06-07 22:30:00  
3700 2025-06-05 01:00:00 2025-06-05 03:00:00  
3701 2016-09-04 14:00:00 2016-09-04 14:50:00  

[3702 rows x 6 columns]

In [34]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2]
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15]
       )
D=pd.DataFrame(D)

In [35]:
C = pd.concat([C]*20_000)
D = pd.concat([D]*1_000)

In [30]:
f=(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
 # ('unitsSold','unitsSold','<'),
    use_numba=True)
)
f

KeyboardInterrupt: 

In [ ]:
f=(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
  ('unitsSold','unitsSold','<'),
 # ('vol','profit','>'),
    use_numba=True)
)
f

left                      right                     
           key vol profit unitsSold   key vol profit unitsSold
0           c1  35     45        15    d5  40     35        40
1           c1  35     45        15    d5  40     35        40
2           c1  35     45        15    d5  40     35        40
3           c1  35     45        15    d5  40     35        40
4           c1  35     45        15    d5  40     35        40
...        ...  ..    ...       ...   ...  ..    ...       ...
199999995   c7  17     11         2    d2  50     10        35
199999996   c7  17     11         2    d2  50     10        35
199999997   c7  17     11         2    d2  50     10        35
199999998   c7  17     11         2    d2  50     10        35
199999999   c7  17     11         2    d2  50     10        35

[200000000 rows x 8 columns]

In [ ]:
f=(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
  ('unitsSold','unitsSold','<'),
  ('vol','profit','>'),
    use_numba=True)
)
f

left                      right                     
          key vol profit unitsSold   key vol profit unitsSold
0          c1  35     45        15    d2  50     10        35
1          c1  35     45        15    d2  50     10        35
2          c1  35     45        15    d2  50     10        35
3          c1  35     45        15    d2  50     10        35
4          c1  35     45        15    d2  50     10        35
...       ...  ..    ...       ...   ...  ..    ...       ...
99999995   c7  17     11         2    d2  50     10        35
99999996   c7  17     11         2    d2  50     10        35
99999997   c7  17     11         2    d2  50     10        35
99999998   c7  17     11         2    d2  50     10        35
99999999   c7  17     11         2    d2  50     10        35

[100000000 rows x 8 columns]

In [ ]:
C.shape[0] * D.shape[0]

1120000000

In [ ]:
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
  #  ('unitsSold','unitsSold','<'),
    use_numba=False)
)

ladiladila


left                      right                     
           key vol profit unitsSold   key vol profit unitsSold
0           c1  35     45        15    d2  50     10        35
1           c1  35     45        15    d2  50     10        35
2           c1  35     45        15    d2  50     10        35
3           c1  35     45        15    d2  50     10        35
4           c1  35     45        15    d2  50     10        35
...        ...  ..    ...       ...   ...  ..    ...       ...
339999995   c7  17     11         2    d2  50     10        35
339999996   c7  17     11         2    d2  50     10        35
339999997   c7  17     11         2    d2  50     10        35
339999998   c7  17     11         2    d2  50     10        35
339999999   c7  17     11         2    d2  50     10        35

[340000000 rows x 8 columns]

In [ ]:
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    ('unitsSold','unitsSold','<'),
    use_numba=False)
)

ladiladila


left                      right                     
           key vol profit unitsSold   key vol profit unitsSold
0           c1  35     45        15    d2  50     10        35
1           c1  35     45        15    d2  50     10        35
2           c1  35     45        15    d2  50     10        35
3           c1  35     45        15    d2  50     10        35
4           c1  35     45        15    d2  50     10        35
...        ...  ..    ...       ...   ...  ..    ...       ...
199999995   c7  17     11         2    d2  50     10        35
199999996   c7  17     11         2    d2  50     10        35
199999997   c7  17     11         2    d2  50     10        35
199999998   c7  17     11         2    d2  50     10        35
199999999   c7  17     11         2    d2  50     10        35

[200000000 rows x 8 columns]

In [ ]:
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
  ('unitsSold','unitsSold','<'),
  ('vol','profit','>'),
    use_numba=False)
)

ladiladila


left                      right                     
          key vol profit unitsSold   key vol profit unitsSold
0          c1  35     45        15    d2  50     10        35
1          c1  35     45        15    d2  50     10        35
2          c1  35     45        15    d2  50     10        35
3          c1  35     45        15    d2  50     10        35
4          c1  35     45        15    d2  50     10        35
...       ...  ..    ...       ...   ...  ..    ...       ...
99999995   c7  17     11         2    d2  50     10        35
99999996   c7  17     11         2    d2  50     10        35
99999997   c7  17     11         2    d2  50     10        35
99999998   c7  17     11         2    d2  50     10        35
99999999   c7  17     11         2    d2  50     10        35

[100000000 rows x 8 columns]

In [36]:
%%timeit
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    ('unitsSold','unitsSold','<'),
    use_numba=True)
)

4.13 s ± 94.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    ('unitsSold','unitsSold','<'),
    use_numba=False)
)

9.78 s ± 36.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
D

,key,vol,profit,unitsSold
0,d1,20,30,20
1,d2,50,10,35
2,d3,15,12,10
3,d4,16,52,12
4,d5,40,35,40
...,...,...,...,...
3,d4,16,52,12
4,d5,40,35,40
5,d6,20,20,30
6,d7,40,30,5


In [ ]:
C.sort_values('vol')

,key,vol,profit,unitsSold
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
...,...,...,...,...
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80


In [ ]:
D.sort_values('vol')
#D.sort_values('profit')
#D.sort_values('profit',ascending=False)


,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35


In [ ]:
D

,key,vol,profit,unitsSold
0,d1,20,30,20
1,d2,50,10,35
2,d3,15,12,10
3,d4,16,52,12
4,d5,40,35,40
...,...,...,...,...
3,d4,16,52,12
4,d5,40,35,40
5,d6,20,20,30
6,d7,40,30,5


In [ ]:
C.sort_values('vol')

,key,vol,profit,unitsSold
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
...,...,...,...,...
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80


In [ ]:
D.sort_values('vol')

,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35


In [ ]:
pd.concat([C.loc[:, ['key','vol']], D.loc[:, ['key','vol']]]).sort_values('vol',kind='stable')

,key,vol
7,d8,2
7,d8,2
7,d8,2
7,d8,2
7,d8,2
...,...,...
5,c6,90
5,c6,90
5,c6,90
5,c6,90


In [ ]:
C.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
2,c3,5,55,30
5,c6,90,55,80
2,c3,5,55,30
5,c6,90,55,80
5,c6,90,55,80
...,...,...,...,...
6,c7,17,11,2
6,c7,17,11,2
6,c7,17,11,2
6,c7,17,11,2


In [ ]:
D.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35


In [ ]:
C.vol.sort_values()

2     5
2     5
2     5
2     5
2     5
     ..
5    90
5    90
5    90
5    90
5    90
Name: vol, Length: 140000, dtype: int64

In [ ]:
C.vol.sort_values().searchsorted(D.vol, side='left')

array([ 80000, 120000,  20000, ...,  80000, 120000,      0])

In [ ]:
D.sort_values(['vol','profit', 'unitsSold'])

,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
